In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.modelbridge.modelbridge_utils import extract_search_space_digest
from ax.models.torch_base import TorchOptConfig

from src.feature_importance import get_model_optimum_in_transformed_space, get_model_optimum_in_original_space

In [3]:
# Setup Ax client
ax_client = AxClient()
ax_client.create_experiment(
    name="test1",
    parameters=[
        {
            "name": "param" + str(feature_idx),
            "type": "range",
            "bounds": [-1.0, 1.0],
            "value_type": "float",
        }
        for feature_idx in range(10)
    ],
    objectives={
        "score": ObjectiveProperties(minimize=False),
    }
)

[INFO 02-13 13:28:13] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-13 13:28:13] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='param0', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param1', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param2', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param3', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param4', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param5', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param6', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param7', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param8', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param9',

In [4]:
# Initialize fixed random Gaussian centers and covariances
_centers = [
    np.random.randn(10) * 0.5,  # Center 1
    np.random.randn(10) * 0.5,  # Center 2
    np.random.randn(10) * 0.5   # Center 3
]

# Create fixed random positive definite covariance matrices
_covs = []
for _ in range(3):
    A = np.random.randn(10, 10)
    cov = A.T @ A + np.eye(10)  # Ensure positive definite
    _covs.append(cov * 0.3)  # Scale down to avoid too sharp peaks

def score(parameterization):
    # Convert parameterization to numpy array
    x = np.array([parameterization[f"param{i}"] for i in range(10)])
    
    # Sum up the Gaussians using fixed centers and covariances
    result = 0
    for center, cov in zip(_centers, _covs):
        diff = x - center
        result += np.exp(-0.5 * diff @ np.linalg.solve(cov, diff))
    
    return {"score": float(result)}


In [5]:
for _ in range(50):
    parameterization, trial_index = ax_client.get_next_trial()
    print(score(parameterization))
    ax_client.complete_trial(trial_index, raw_data=score(parameterization))

ax_client.get_best_parameters()

/Users/keenanpepper/ax-feature-importance/.venv/lib/python3.12/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 02-13 13:28:17] ax.service.ax_client: Generated new trial 0 with parameters {'param0': 0.732023, 'param1': -0.400775, 'param2': -0.366706, 'param3': -0.537021, 'param4': 0.955392, 'param5': -0.358135, 'param6': 0.728747, 'param7': -0.275478, 'param8': -0.095125, 'param9': -0.72373} using model Sobol.
[INFO 02-13 13:28:17] ax.service.ax_client: Completed trial 0 with data: {'score': (0.442373, None)}.
/Users/keenanpepper/ax-feature-importance/.venv/lib/python3.12/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit q

{'score': 0.4423733479796191}
{'score': 1.1768845000155201}
{'score': 0.31749223902826007}
{'score': 0.14001284867893102}
{'score': 0.2778520098509579}
{'score': 0.47958923600355907}
{'score': 0.8258814413725954}
{'score': 0.31697474588259533}
{'score': 0.09212616820068063}
{'score': 0.1908870702970132}
{'score': 0.6373027442467919}
{'score': 0.5194261994145237}
{'score': 0.28536263780032783}
{'score': 0.8553719485062397}
{'score': 0.17179445951324746}
{'score': 0.39188595835137147}
{'score': 0.1501517088872938}
{'score': 0.6933007550810226}
{'score': 1.3695671609897002}
{'score': 0.7739556253003195}


[INFO 02-13 13:28:18] ax.service.ax_client: Generated new trial 20 with parameters {'param0': -0.748639, 'param1': -0.063618, 'param2': 0.286315, 'param3': 0.262247, 'param4': 0.176234, 'param5': 0.196628, 'param6': -0.509886, 'param7': -0.446305, 'param8': 0.929357, 'param9': 0.877951} using model BoTorch.
[INFO 02-13 13:28:18] ax.service.ax_client: Completed trial 20 with data: {'score': (1.122799, None)}.


{'score': 1.1227985051551568}


[INFO 02-13 13:28:19] ax.service.ax_client: Generated new trial 21 with parameters {'param0': -0.404091, 'param1': 0.308286, 'param2': 0.113787, 'param3': 0.441603, 'param4': -0.49446, 'param5': 0.32134, 'param6': -0.497365, 'param7': -0.213975, 'param8': 0.478353, 'param9': 0.723916} using model BoTorch.
[INFO 02-13 13:28:19] ax.service.ax_client: Completed trial 21 with data: {'score': (1.176923, None)}.


...

[INFO 02-13 13:28:35] ax.service.ax_client: Generated new trial 48 with parameters {'param0': -0.492545, 'param1': -0.180101, 'param2': 0.28842, 'param3': -0.274932, 'param4': -0.273785, 'param5': -0.076986, 'param6': -0.042842, 'param7': -0.277481, 'param8': 0.555709, 'param9': 0.089415} using model BoTorch.
[INFO 02-13 13:28:35] ax.service.ax_client: Completed trial 48 with data: {'score': (2.047967, None)}.


{'score': 2.0479666020603093}


[INFO 02-13 13:28:36] ax.service.ax_client: Generated new trial 49 with parameters {'param0': -0.355377, 'param1': -0.207119, 'param2': 0.342492, 'param3': -0.251601, 'param4': -0.239677, 'param5': -0.055007, 'param6': 0.295656, 'param7': -0.292313, 'param8': 0.661442, 'param9': 0.0992} using model BoTorch.
[INFO 02-13 13:28:36] ax.service.ax_client: Completed trial 49 with data: {'score': (1.916571, None)}.


{'score': 1.9165709644920264}


({'param0': -0.49254545297255725,
  'param1': -0.18010097272708503,
  'param2': 0.28841973299350143,
  'param3': -0.27493176641899153,
  'param4': -0.27378545008663246,
  'param5': -0.0769856852599502,
  'param6': -0.04284196802397078,
  'param7': -0.27748145600102625,
  'param8': 0.5557092004658422,
  'param9': 0.08941505889834067},
 ({'score': np.float64(2.014278037591337)},
  {'score': {'score': np.float64(0.00036798054809794036)}}))

In [9]:
model = ax_client.generation_strategy.model.model  # Get the BoTorchModel
bounds = torch.stack([torch.zeros(10), torch.ones(10)])  # Create bounds tensor
optimum = get_model_optimum_in_transformed_space(model, bounds, fixed_features={0: (-0.4925 + 1) / 2})
optimum

tensor([0.2537, 0.4043, 0.6564, 0.3693, 0.3696, 0.4604, 0.5152, 0.3570, 0.7966,
        0.5502])

In [10]:
optimum * 2 - 1

tensor([-0.4925, -0.1914,  0.3129, -0.2613, -0.2608, -0.0792,  0.0304, -0.2860,
         0.5933,  0.1004])

In [11]:
ax_client.get_best_parameters()

({'param0': -0.49254545297255725,
  'param1': -0.18010097272708503,
  'param2': 0.28841973299350143,
  'param3': -0.27493176641899153,
  'param4': -0.27378545008663246,
  'param5': -0.0769856852599502,
  'param6': -0.04284196802397078,
  'param7': -0.27748145600102625,
  'param8': 0.5557092004658422,
  'param9': 0.08941505889834067},
 ({'score': np.float64(2.014278037591337)},
  {'score': {'score': np.float64(0.00036798054809794036)}}))

In [16]:
ax_client.experiment.search_space

SearchSpace(parameters=[RangeParameter(name='param0', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param1', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param2', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param3', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param4', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param5', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param6', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param7', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param8', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param9', parameter_type=FLOAT, range=[-1.0, 1.0])], parameter_constraints=[])

In [47]:
get_model_optimum_in_original_space(ax_client, fixed_features={"param9": 0.089})

{'param0': -0.45936959981918335,
 'param1': -0.19075411558151245,
 'param2': 0.3131176233291626,
 'param3': -0.2619051933288574,
 'param4': -0.25843995809555054,
 'param5': -0.07730764150619507,
 'param6': 0.034264206886291504,
 'param7': -0.2877769470214844,
 'param8': 0.5947976112365723,
 'param9': 0.08899998664855957}

In [44]:
ax_client.get_best_parameters()

({'param0': -0.49254545297255725,
  'param1': -0.18010097272708503,
  'param2': 0.28841973299350143,
  'param3': -0.27493176641899153,
  'param4': -0.27378545008663246,
  'param5': -0.0769856852599502,
  'param6': -0.04284196802397078,
  'param7': -0.27748145600102625,
  'param8': 0.5557092004658422,
  'param9': 0.08941505889834067},
 ({'score': np.float64(2.014278037591337)},
  {'score': {'score': np.float64(0.00036798054809794036)}}))